In [2]:
# import dependencies
import pandas as pd
import gmaps
import requests

# import api key
from config import g_key

In [3]:
# store the CSV you saved created in part one into a DataFrame
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Semey,KZ,2022-07-18 17:54:29,50.4111,80.2275,67.01,42,40,3.67
1,1,Filingue,NE,2022-07-18 17:54:30,14.3521,3.3168,87.17,55,63,13.73
2,2,Cabo San Lucas,MX,2022-07-18 17:49:47,22.8909,-109.9124,92.61,65,27,5.01
3,3,Doka,SD,2022-07-18 17:54:30,13.5167,35.7667,78.31,75,75,16.60
4,4,Kapaa,US,2022-07-18 17:51:37,22.0752,-159.3190,77.32,89,100,16.11


In [4]:
# get data types
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [5]:
# configure gmaps to use your Google API key
gmaps.configure(api_key=g_key)

In [6]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [19]:
# Heatmap of temperature

# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps,
                                 dissipating=False, max_intensity=230, 
                                 point_radius=6)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 73
What is the maximum temperature you would like for your trip? 90


In [26]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Filingue,NE,2022-07-18 17:54:30,14.3521,3.3168,87.17,55,63,13.73
3,3,Doka,SD,2022-07-18 17:54:30,13.5167,35.7667,78.31,75,75,16.60
4,4,Kapaa,US,2022-07-18 17:51:37,22.0752,-159.3190,77.32,89,100,16.11
15,15,San Cristobal,VE,2022-07-18 17:50:11,7.7669,-72.2250,78.98,71,88,6.17
17,17,Kirakira,SB,2022-07-18 17:54:35,-10.4544,161.9205,78.49,84,32,6.26
20,20,Ponta Do Sol,PT,2022-07-18 17:50:22,32.6667,-17.1000,76.01,76,13,3.85
21,21,Vaini,TO,2022-07-18 17:54:36,-21.2000,-175.2000,73.56,88,20,21.85
22,22,Requena,ES,2022-07-18 17:54:36,39.4883,-1.1004,84.79,31,0,12.28
23,23,Mazamari,PE,2022-07-18 17:54:36,-11.3258,-74.5308,81.91,63,95,2.04
24,24,Lorengau,PG,2022-07-18 17:54:37,-2.0226,147.2712,79.90,77,100,4.83


In [28]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Filingue,NE,87.17,14.3521,3.3168,
3,Doka,SD,78.31,13.5167,35.7667,
4,Kapaa,US,77.32,22.0752,-159.3190,
15,San Cristobal,VE,78.98,7.7669,-72.2250,
17,Kirakira,SB,78.49,-10.4544,161.9205,
20,Ponta Do Sol,PT,76.01,32.6667,-17.1000,
21,Vaini,TO,73.56,-21.2000,-175.2000,
22,Requena,ES,84.79,39.4883,-1.1004,
23,Mazamari,PE,81.91,-11.3258,-74.5308,
24,Lorengau,PG,79.90,-2.0226,147.2712,


In [29]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [31]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print('Hotel not found...skipping')

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


In [33]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.05)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=250, point_radius=5)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [34]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [36]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.05)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=250, point_radius=5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))